In [1]:

#### Import required dependencies
import pandas as pd
from prophet import Prophet
import datetime as dt
import numpy as np
%matplotlib inline

In [3]:

#### Read in the datasets
household_size_data = pd.read_csv("Resources/avg-household-size.csv", index_col='geography')
#household_size_data.info()
household_size_data.head()


,index,statefips,countyfips,avghouseholdsize
geography,,,,
"Aleutians East Borough, Alaska",0,2,13,2.43
"Aleutians West Census Area, Alaska",1,2,16,3.59
"Anchorage Municipality, Alaska",2,2,20,2.77
"Bethel Census Area, Alaska",3,2,50,3.86
"Bristol Bay Borough, Alaska",4,2,60,2.50


In [5]:

##
cancer_data = pd.read_csv("Resources/cancer_reg.csv", index_col='geography')
#cancer_data.info()
cancer_data.head()


,index,avganncount,avgdeathsperyear,target_deathrate,incidencerate,medincome,popest2015,povertypercent,studypercap,binnedinc,...,pctprivatecoveragealone,pctempprivcoverage,pctpubliccoverage,pctpubliccoveragealone,pctwhite,pctblack,pctasian,pctotherrace,pctmarriedhouseholds,birthrate
geography,,,,,,,,,,,,,,,,,,,,,
"Kitsap County, Washington",0,1397.0,469,164.9,489.8,61898,260131,11.2,499.748204,"(61494.5, 125635]",...,NaN,41.6,32.9,14.0,81.780529,2.594728,4.821857,1.843479,52.856076,6.118831
"Kittitas County, Washington",1,173.0,70,161.3,411.6,48127,43269,18.6,23.111234,"(48021.6, 51046.4]",...,53.8,43.6,31.1,15.3,89.228509,0.969102,2.246233,3.741352,45.372500,4.333096
"Klickitat County, Washington",2,102.0,50,174.7,349.7,49348,21026,14.6,47.560164,"(48021.6, 51046.4]",...,43.5,34.9,42.1,21.1,90.922190,0.739673,0.465898,2.747358,54.444868,3.729488
"Lewis County, Washington",3,427.0,202,194.8,430.4,44243,75882,17.1,342.637253,"(42724.4, 45201]",...,40.3,35.0,45.3,25.0,91.744686,0.782626,1.161359,1.362643,51.021514,4.603841
"Lincoln County, Washington",4,57.0,26,144.4,350.1,49955,10321,12.5,0.000000,"(48021.6, 51046.4]",...,43.9,35.1,44.0,22.7,94.104024,0.270192,0.665830,0.492135,54.027460,6.796657


In [11]:

#### Combine the dataframes on geo
combined_data = pd.merge(household_size_data, cancer_data, on="geography", how="outer")
combined_data.info()
combined_data.head()


<class 'pandas.core.frame.DataFrame'>
Index: 3220 entries, Aleutians East Borough, Alaska to Weston County, Wyoming
Data columns (total 37 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   index_x                  3220 non-null   int64  
 1   statefips                3220 non-null   int64  
 2   countyfips               3220 non-null   int64  
 3   avghouseholdsize         3220 non-null   float64
 4   index_y                  3047 non-null   float64
 5   avganncount              3047 non-null   float64
 6   avgdeathsperyear         3047 non-null   float64
 7   target_deathrate         3047 non-null   float64
 8   incidencerate            3047 non-null   float64
 9   medincome                3047 non-null   float64
 10  popest2015               3047 non-null   float64
 11  povertypercent           3047 non-null   float64
 12  studypercap              3047 non-null   float64
 13  binnedinc                3047 non-nu

,index_x,statefips,countyfips,avghouseholdsize,index_y,avganncount,avgdeathsperyear,target_deathrate,incidencerate,medincome,...,pctprivatecoveragealone,pctempprivcoverage,pctpubliccoverage,pctpubliccoveragealone,pctwhite,pctblack,pctasian,pctotherrace,pctmarriedhouseholds,birthrate
geography,,,,,,,,,,,,,,,,,,,,,
"Aleutians East Borough, Alaska",0,2,13,2.43,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
"Aleutians West Census Area, Alaska",1,2,16,3.59,2713.0,8.0,3.0,203.3,201.3,68387.0,...,60.4,58.3,11.2,6.2,31.562280,4.732583,37.156932,5.295567,42.882250,4.391052
"Anchorage Municipality, Alaska",2,2,20,2.77,2714.0,1085.0,352.0,167.5,453.3,75200.0,...,NaN,45.9,24.5,12.9,64.944652,5.854427,8.800530,1.747201,48.361303,5.994294
"Bethel Census Area, Alaska",3,2,50,3.86,2715.0,37.0,20.0,218.8,338.2,45808.0,...,25.8,25.1,47.3,36.1,12.016202,0.798830,1.175743,0.343159,45.521390,9.156280
"Bristol Bay Borough, Alaska",4,2,60,2.50,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [15]:

#### Description of columns
# statefips: State identification number. (Integer)
# countyfips: County identification number. (Integer)
# avghouseholdsize: Average size of households in the county. (Float)
# geography: Location name. (String)
# geography: Location name. (String)
# avganncount: Average annual count of cancer cases in the county. (Numeric)
# avgdeathsperyear: Average number of deaths per year in the county. (Numeric)
# target_deathrate: Number of deaths per 100k individuals in each county. (Numeric)
# medincome: Median household income for the county. (Numeric)
# popest2015: Population estimates for 2015. (Numeric)
# povertypercent: Percentage of people living in poverty. (Numeric)
# studypercap: Number of clinical trials per capita in the given county. (Numeric)
# binnedinc: Binned income for each county. (Categorical)
# medianage: Median age of the population in the county. (Numeric)
# medianagemale: Median age of the male population in the county. (Numeric)
# medianagefemale: Median age of the female population in the county. (Numeric)
# percentmarried: Percentage of people who are married in the county. (Numeric)
# pctnohs18_24: Percentage of people aged 18-24 who did not graduate high school. (Numeric)
# pcths18_24: Percentage of people aged 18-24 who graduated high school. (Numeric)
# pctsomecol18_24: Percentage of people aged 18-24 who attended some college. (Numeric)
# pctbachdeg18_24: Percentage of people aged 18-24 who have a bachelor's degree. (Numeric)
# pcths25_over: Percentage of people aged 25 and over who graduated high school. (Numeric)
# pctbachdeg25_over: Percentage of people aged 25 and over who have a bachelor's degree. (Numeric)
# pctemployed16_over: Percentage of people aged 16 and over who are employed. (Numeric)
# pctunemployed16_over: Percentage of people aged 16 and over who are unemployed. (Numeric)
# pctprivatecoverage: Percentage of people with private health insuranc

#### Drop the NA values
combined_data = combined_data.dropna()
combined_data.info()

#### Retain only data we are interested in
combined_data = combined_data.drop(columns=['index_x', 'index_y', 'medincome', 'povertypercent', 'binnedinc', 'percentmarried',
                                            'pctnohs18_24', 'pcths18_24', 'pctsomecol18_24', 'pctbachdeg18_24', 'pcths25_over',
                                           'pctbachdeg25_over', 'pctemployed16_over', 'pctunemployed16_over', 'pctprivatecoverage',
                                           'pctprivatecoveragealone', 'pctempprivcoverage', 'pctpubliccoverage', 'pctpubliccoveragealone',
                                            'pctwhite', 'pctblack', 'pctasian', 'pctotherrace', 'pctmarriedhouseholds', 'birthrate'])
combined_data.info()



<class 'pandas.core.frame.DataFrame'>
Index: 591 entries, Haines Borough, Alaska to Sheridan County, Wyoming
Data columns (total 37 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   index_x                  591 non-null    int64  
 1   statefips                591 non-null    int64  
 2   countyfips               591 non-null    int64  
 3   avghouseholdsize         591 non-null    float64
 4   index_y                  591 non-null    float64
 5   avganncount              591 non-null    float64
 6   avgdeathsperyear         591 non-null    float64
 7   target_deathrate         591 non-null    float64
 8   incidencerate            591 non-null    float64
 9   medincome                591 non-null    float64
 10  popest2015               591 non-null    float64
 11  povertypercent           591 non-null    float64
 12  studypercap              591 non-null    float64
 13  binnedinc                591 non-null    ob

In [17]:
combined_data

,statefips,countyfips,avghouseholdsize,avganncount,avgdeathsperyear,target_deathrate,incidencerate,popest2015,studypercap,medianage,medianagemale,medianagefemale
geography,,,,,,,,,,,,
"Haines Borough, Alaska",2,100,2.12,13.0,4.0,126.3,413.3,2534.0,0.000000,45.5,45.6,45.3
"Kenai Peninsula Borough, Alaska",2,122,2.57,266.0,96.0,168.1,441.4,58059.0,0.000000,40.5,40.0,41.1
"Ketchikan Gateway Borough, Alaska",2,130,2.55,63.0,25.0,205.0,477.3,13709.0,0.000000,38.7,37.9,39.4
"Yukon-Koyukuk Census Area, Alaska",2,290,2.81,27.0,11.0,206.6,475.1,5533.0,0.000000,35.0,35.4,34.0
"Cherokee County, Alabama",1,19,2.28,158.0,69.0,191.2,432.6,25859.0,0.000000,45.4,44.6,46.2
...,...,...,...,...,...,...,...,...,...,...,...,...
"Putnam County, West Virginia",54,79,2.60,328.0,114.0,171.5,497.2,56848.0,211.089220,41.0,40.4,41.6
"Webster County, West Virginia",54,101,2.28,65.0,33.0,259.5,511.9,8755.0,114.220445,46.3,45.5,46.6
"Hot Springs County, Wyoming",56,17,2.15,37.0,13.0,163.0,484.2,4741.0,0.000000,49.6,48.1,50.1
